CTE (Common Table Expressions)

In [ ]:
WITH vendas_recentes AS (
    SELECT cliente_id, valor
    FROM vendas
    WHERE data >= '2024-01-01'
)
SELECT cliente_id, SUM(valor) as total
FROM vendas_recentes
GROUP BY cliente_id;


NULL

In [ ]:
--- Selecting when is not null
SELECT * FROM table WHERE id ID NOT NULL

In [ ]:
--- Replacing missing values
SELECT COALESCE(price,0) AS clean_price FROM sales

In [ ]:
--- Replacing using CASE WHEN
SELECT
    CASE
        WHEN region IS NULL THEN 'Unknown'
        ELSE region
    END AS region_cleaned
FROM users

Window Function

In [ ]:
--- Row_number gives a unique row number per group
--- Use case: Get the top 1 salary per department
SELECT 
    employee_id,
    department,
    salary,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS rank_in_dept
FROM employees

In [ ]:
--- RANK / DENSE_RANK - rank with out without gaps
SELECT
    product_id,
    category,
    sales,
    RANK() OVER (PARTITION BY category ORDER BY sales DESC) AS sales_rank
FROM products

In [ ]:
--- LAG/LEAD - compare values in adjacent rows
--- Use case: Track how a customer's spending changes over time.
SELECT
    customer_id,
    purschse_date,
    amount,
    LAG(amount) OVER (PARTITION BY customer_id ORDER BY purchse_date) AS prev_purchase,
    LEAD(amount) OVER (PARTITION BY customer_id ORDER BY purchase_date) AS next_purchase
FROM purchases

In [ ]:
--- SUm/AVG/MAX 
--- Use case: Rolling totals or moving averages
SELECT 
     order_id,
     customer_id,
     order_value,
     SUM(order_value) OVER (PARTITION BY customer_id ORDER BY order_date ROWS BETWEEN 3 PRECEDING AND CURRENT ROW ) AS rolling_3_sum
FROM orders

In [ ]:
--- NTILE - break rows into buckets
--- Use case: Quartile or deciles of scores

SELECT 
    student_id,
    test_score,
    NTILE(4) OVER (ORDER BY test_score DESC) AS quartile
FROM students

Select date


In [ ]:
SELECT user_id COUNT(app_id) AS total_downloads
from isntalls
WHERE intall_date BETWEEN date_sub(CURRENT_DATE, INTERVAL 7 DAYS) AND CURRENT_DATE
GROUP BY user_id

In [ ]:
WITH users_purchases AS (
  SELECT u.user_id, u.signup_date, p.purchase_amount
  FROM users u
  LEFT JOIN purchases p ON u.user_id = p.user_id
)
SELECT user_id, SUM(purchase_amount) AS total_purchase
FROM users_purchases
WHERE signup_date >= CURRENT_DATE - INTERVAL 30 DAY
GROUP BY user_id;


Duplicates

In [ ]:
SELECT user_id, COUNT(*) as num_duplicates
FROM users
GROUP BY user_id 
HAVING COUNT(*) > 1

Example of a complex query

finding the top 10 turbine makes by number of inspections per U.S. state over the last 2 years, considering only severity > 5:

inspection (inspection_id, inspection_date, blade_id, severity)

blade (blade_id, turbine_id)

turbine (turbine_id, customer_id, site_id)

make_model (turbine_id, make)

site (site_id, state)

In [ ]:
WITH filtered_inspections AS (
    SELECT 
        i.inspection_id,
        i.inspection_date,
        i.severity,
        b.turbine_id
    FROM inspection i
    JOIN blade b ON i.blade_id = b.blade_id
    WHERE i.severity > 5
      AND i.inspection_date >= DATEADD(YEAR, -2, CURRENT_DATE)
),

joined_data AS (
    SELECT 
        fi.inspection_id,
        fi.turbine_id,
        t.site_id,
        mm.make
    FROM filtered_inspections fi
    JOIN turbine t ON fi.turbine_id = t.turbine_id
    JOIN make_model mm ON fi.turbine_id = mm.turbine_id
),

final_data AS (
    SELECT 
        jd.inspection_id,
        jd.make,
        s.state
    FROM joined_data jd
    JOIN site s ON jd.site_id = s.site_id
),

inspections_per_make_state AS (
    SELECT 
        state,
        make,
        COUNT(DISTINCT inspection_id) AS total_inspections,
        ROW_NUMBER() OVER (PARTITION BY state ORDER BY COUNT(DISTINCT inspection_id) DESC) AS rank
    FROM final_data
    GROUP BY state, make
)

SELECT 
    state,
    make,
    total_inspections
FROM inspections_per_make_state
WHERE rank <= 10
ORDER BY state, total_inspections DESC;


In [ ]:
SELECT categoria, valor
FROM (
    SELECT 
        categoria,
        valor,
        DENSE_RANK() OVER (PARTITION BY categoria ORDER BY valor DESC) AS rk
    FROM vendas
) AS sub
WHERE rk <= 3;